# Sync Code

Clone the GitHub Repo

In [1]:
!cd /content
!git clone https://github.com/szappac20/training_camp_21_22.git -b develop

Cloning into 'training_camp_21_22'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 39 (delta 13), reused 31 (delta 8), pack-reused 0
Unpacking objects: 100% (39/39), done.


Move to the Python Project root folder and install requirements

In [2]:
%cd /content/training_camp_21_22
%pip install --upgrade -r requirements.txt -qqq

/content/training_camp_21_22
     |████████████████████████████████| 38.1 MB 1.6 MB/s 
     |████████████████████████████████| 9.8 MB 27.2 MB/s 
     |████████████████████████████████| 511.7 MB 6.0 kB/s 
     |████████████████████████████████| 554 kB 72.3 MB/s 
     |████████████████████████████████| 13.1 MB 19.2 MB/s 
     |████████████████████████████████| 1.1 MB 44.3 MB/s 
     |████████████████████████████████| 15.8 MB 36.4 MB/s 
     |████████████████████████████████| 1.6 MB 52.2 MB/s 
     |████████████████████████████████| 438 kB 76.0 MB/s 
     |████████████████████████████████| 5.8 MB 50.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


# Download Dataset from Kaggle competition

Download dataset from Kaggle competition pitc-2122-prediction

In [3]:
!mkdir -p ~/.kaggle

Download the kaggle.json token file

In [4]:
!gdown 1w81nWRhAr1E_hX9OSncuMxlCefxnP2zN -O  ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

Downloading...
From: https://drive.google.com/uc?id=1w81nWRhAr1E_hX9OSncuMxlCefxnP2zN
To: /root/.kaggle/kaggle.json
100% 74.0/74.0 [00:00<00:00, 117kB/s]


Download the input file (training and test)

In [5]:
# !mkdir -p data/task_1
!mkdir -p data/task_2

In [6]:

# %cd data/task_1

In [7]:
# !kaggle competitions download -c pitc-2122-clustering
# !unzip pitc-2122-clustering.zip

In [8]:
%cd data/task_2

/content/training_camp_21_22/data/task_2


In [9]:
!kaggle competitions download -c pitc-2122-prediction
!unzip pitc-2122-prediction.zip

 82% 65.0M/79.3M [00:00<00:00, 96.8MB/s]
100% 79.3M/79.3M [00:00<00:00, 105MB/s] 
Archive:  pitc-2122-prediction.zip
  inflating: solution_sample.csv     
  inflating: test.csv                
  inflating: train.csv               


In [10]:
%cd /content/training_camp_21_22

/content/training_camp_21_22


# Import packages

Move back to the Python project folder

In [11]:
import sys
import os

sys.path.append("training_camp_21_22")

Import libraries

In [12]:
import datetime
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import itertools

import training_camp_21_22.config
import training_camp_21_22.models

import training_camp_21_22.models.baseline
import training_camp_21_22.data.window_generator

# Preprocess Data

### **Train Data**

In [14]:
# Parse the csv file with json entries: Train Data
import json

def convert_json_2_pd(json_input_path):
    """
    Read the csv file and extract from json events
    any feature useful to classify the consumer consumption profile 
    """
    df=pd.read_csv(json_input_path)
    l=[]
    for i in range(len(df)):
          
          event = df["events"][i]
          lst=[]
          row_data = json.loads(event)
          lst.append(df["user_month_year"][i])

          # bamData.items.bamItem.addresses: List of Address
          lst.append(row_data[0]["bamData"]["items"]["bamItem"]["personInfo"]["birthDate"])
          lst.append(row_data[0]["bamData"]["items"]["bamItem"]["addresses"][1]['zipCode']['code'])

          # bamData.items.customData.energy: Energy Consumption related for the period in scope
          lst.append( row_data[0]["customData"]["energy"]["fasciaF1ConsumoFatturato"])
          lst.append(row_data[0]["customData"]["energy"]["fasciaF2ConsumoFatturato"])
          lst.append(row_data[0]["customData"]["energy"]["fasciaF3ConsumoFatturato"])
          l.append(lst)

    features_df = pd.DataFrame(l, columns =['user_month_year','age','location','f1','f2','f3'], dtype = float)  
    features_df['next_month_f1']=df['next_month_f1']
    features_df['next_month_f2']=df['next_month_f2']
    features_df['next_month_f3']=df['next_month_f3']

    return features_df

train_df  = convert_json_2_pd(json_input_path='/content/training_camp_21_22/data/task_2/train.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised


In [15]:
train_df.shape

(770000, 9)

In [16]:
train_df.head(2)

,user_month_year,age,location,f1,f2,f3,next_month_f1,next_month_f2,next_month_f3
0,72022.0,1980-07-06,40051.0,56.0,42.0,60.0,59.0,44.0,57.0
1,82022.0,1980-07-06,40051.0,59.0,44.0,57.0,32.0,31.0,37.0


### **Test Data**

In [17]:
# Parse the csv file with json entries :Test data
import json

def convert_json_2_pd(json_input_path):
    """
    Read the csv file and extract from json events
    any feature useful to classify the consumer consumption profile 
    """
    df=pd.read_csv(json_input_path)
    l=[]
    for i in range(len(df)):
          
          event = df["events"][i]
          lst=[]
          row_data = json.loads(event)
          lst.append(df["user_month_year"][i])

          # bamData.items.bamItem.addresses: List of Address
          lst.append(row_data[0]["bamData"]["items"]["bamItem"]["personInfo"]["birthDate"])
          lst.append(row_data[0]["bamData"]["items"]["bamItem"]["addresses"][1]['zipCode']['code'])

          # bamData.items.customData.energy: Energy Consumption related for the period in scope
          lst.append( row_data[0]["customData"]["energy"]["fasciaF1ConsumoFatturato"])
          lst.append(row_data[0]["customData"]["energy"]["fasciaF2ConsumoFatturato"])
          lst.append(row_data[0]["customData"]["energy"]["fasciaF3ConsumoFatturato"])
          l.append(lst)

    features_df = pd.DataFrame(l, columns =['user_month_year','age','location','f1','f2','f3'], dtype = float)  

    return features_df

test_df  = convert_json_2_pd(json_input_path='/content/training_camp_21_22/data/task_2/test.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised


In [18]:
test_df.head(2)

,user_month_year,age,location,f1,f2,f3
0,9.000007e+10,2002-06-04,96010.0,0.0,34.0,33.0
1,9.000008e+10,2002-06-04,96010.0,0.0,25.0,35.0


In [19]:
test_df.shape

(110000, 6)

Fill the missing data (for the data coming from the past history)

In [20]:
train_df.isnull().sum()


user_month_year    0
age                0
location           0
f1                 0
f2                 0
f3                 0
next_month_f1      0
next_month_f2      0
next_month_f3      0
dtype: int64

In [21]:
test_df.isnull().sum()


user_month_year    0
age                0
location           0
f1                 0
f2                 0
f3                 0
dtype: int64

### **Data Elimination**

## **Train**

In [31]:
df_train=train_df

In [32]:
# Drop rows with wrong first column(user_month_year)
df_train=df_train[df_train.user_month_year>=9999999999]
df_train.shape

(660000, 9)

In [1]:
# change type of column
df_train['user_month_year'] = df_train['user_month_year'].apply(np.int64)

df_train['user_month_year'] = df_train['user_month_year'].apply(str)

# Splite first column to 3 columns
df_train['year'] = df_train['user_month_year'].str[-4:]
df_train['year'] = df_train['year'].apply(np.int64)

df_train['month'] = df_train['user_month_year'].astype(str).str[-6:-4]
df_train['month'] = df_train['month'].apply(np.int64)

df_train['user'] = df_train['user_month_year'].astype(str).str[:-6]
df_train['user'] = df_train['user'].apply(np.int64)


df_train=df_train.drop('user_month_year',axis=1)

df_train['age'] =pd.to_datetime(df_train['age'])
import datetime as dt
now_year =dt.datetime.today().year 
df_train['age']=now_year-df_train.age.dt.year

df_train=df_train.reset_index(drop=True)

In [34]:
df_train.head(2)

,age,location,f1,f2,f3,next_month_f1,next_month_f2,next_month_f3,year,month,user
0,32,72100.0,77.0,40.0,79.0,70.0,58.0,72.0,2022,7,10000
1,32,72100.0,70.0,58.0,72.0,54.0,72.0,63.0,2022,8,10000


In [35]:
df_train.shape

(660000, 11)

# **Test**

In [44]:
# Drop rows with wrong first column(user_month_year)
df_test=test_df[test_df.user_month_year>=9999999999]

df_test.shape

(110000, 6)

In [45]:
# change type of column
import datetime as dt

df_test['user_month_year'] = df_test['user_month_year'].apply(np.int64)

df_test['user_month_year'] = df_test['user_month_year'].apply(str)

# Splite first column to 3 columns
df_test['year'] = df_test['user_month_year'].str[-4:]
df_test['year'] = df_test['year'].apply(np.int64)

df_test['month'] = df_test['user_month_year'].astype(str).str[-6:-4]
df_test['month'] = df_test['month'].apply(np.int64)

df_test['user'] = df_test['user_month_year'].astype(str).str[:-6]
df_test['user'] = df_test['user'].apply(np.int64)

df_test=df_test.drop('user_month_year',axis=1)

df_test['age'] =pd.to_datetime(df_test['age'])
now_year =dt.datetime.today().year 
df_test['age']=now_year-df_test.age.dt.year

df_test=df_test.reset_index(drop=True)

In [46]:
df_test.head(2)

,age,location,f1,f2,f3,year,month,user
0,20,96010.0,0.0,34.0,33.0,2022,7,90000
1,20,96010.0,0.0,25.0,35.0,2022,8,90000


# Model

In [47]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# create dataset
X=df_train[['age','location','f1', 'f2', 'f3','year', 'month', 'user']]
y=df_train[['next_month_f1', 'next_month_f2', 'next_month_f3']]
scaler = MinMaxScaler()
X= scaler.fit_transform(X)
test = scaler.transform(df_test)

In [48]:
X.shape , y.shape, test.shape

((660000, 8), (660000, 3), (110000, 8))

In [49]:
# example of evaluating direct multioutput regression with an SVM model
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import LinearSVR
model = LinearSVR()

wrapper = MultiOutputRegressor(model)

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

n_scores = cross_val_score(wrapper, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

n_scores = absolute(n_scores)
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


MAE: 7.246 (0.020)


In [50]:
# Prediction Testset
wrapper = MultiOutputRegressor(model)
wrapper.fit(X, y)
y_test_multi = wrapper.predict(test)

In [71]:
baseline_results_multi_reg=pd.DataFrame(y_test_multi, columns =['next_month_f1', 'next_month_f2', 'next_month_f3'], dtype = float) 
baseline_results_multi_reg['user_month_year']=test_df['user_month_year']
baseline_results_multi_reg.head(2)


,next_month_f1,next_month_f2,next_month_f3,user_month_year
0,-0.121648,31.019431,33.749582,9.000007e+10
1,0.078704,22.935222,35.732555,9.000008e+10


In [72]:
baseline_results_multi_reg['user_month_year'] = baseline_results_multi_reg['user_month_year'].apply(np.int64)
baseline_results_multi_reg['user_month_year'] = baseline_results_multi_reg['user_month_year'].apply(str)
baseline_results_multi_reg['user_month_year']=baseline_results_multi_reg['user_month_year'].str[:-6]+"_"+ baseline_results_multi_reg['user_month_year'].str[-6:-4]+"_"+ baseline_results_multi_reg['user_month_year'].str[-4:]
baseline_results_multi_reg.head()


,next_month_f1,next_month_f2,next_month_f3,user_month_year
0,-0.121648,31.019431,33.749582,90000_07_2022
1,0.078704,22.935222,35.732555,90000_08_2022
2,2.204059,18.548344,34.077740,90000_09_2022
3,4.182087,11.430358,40.623270,90000_10_2022
4,3.518155,3.318222,38.958241,90000_11_2022


In [73]:
baseline_results_multi_reg.to_csv('baseline_results_multi_reg.csv',index=False)